In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/lib/kaggle/gcp.py
/kaggle/input/custom/test_data_nomentions_sanitized.csv
/kaggle/input/custom/train_data_noemoji_nomentions.csv
/kaggle/input/nitro-language-processing-2/sample_submission.csv
/kaggle/input/nitro-language-processing-2/random_seed_setter.py
/kaggle/input/nitro-language-processing-2/train_data.csv
/kaggle/input/nitro-language-processing-2/test_data.csv
/kaggle/input/nitro-language-processing-2/random_seeds.csv
/kaggle/working/__notebook_source__.ipynb


In [7]:
import torch
import torch.nn as nn
import pandas as pd
from transformers import AutoTokenizer, BertForSequenceClassification
from torch.utils.data import Dataset
import wandb
import shutil
from torch.utils.data import WeightedRandomSampler
import os
from sklearn.metrics import balanced_accuracy_score

In [8]:
class SexismDetectionModel(nn.Module):
    def __init__(self, model_name, num_classes):
        super().__init__()
        self.bert_model = BertForSequenceClassification.from_pretrained(model_name, num_labels=num_classes,
                                                                        return_dict=False)

    def forward(self, x):
        return self.bert_model(x)

In [9]:
class SexismDataset(Dataset):

    def __init__(self, model_name,  file, test=False):
        data = pd.read_csv(file)
        self.text = data["Text"].tolist()
        self.label_mapping = {
            'direct': 0,
            'descriptive': 1,
            'reporting': 2,
            'offensive': 3,
            'non-offensive': 4
        }
        self.padding = 64
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.test = test
        self.maximum = 0
        self.count = 0
        if not test:
            self.labels = data["Final Labels"].tolist()

    def __len__(self):
        return len(self.text)

    def __getitem__(self, item):

         
        input_ids = self.tokenizer.encode(self.text[item], add_special_tokens=True, return_tensors="pt",
                                          max_length=self.padding,
                                          padding='max_length',
                                          truncation=True).view(-1)
        if not self.test:
            return input_ids, self.label_mapping[self.labels[item]]
        return input_ids

In [10]:
def save_ckp(state, is_best, checkpoint_dir, best_model_dir):
    f_path = checkpoint_dir + '/checkpoint.pt'
    torch.save(state, f_path)
    if is_best:
        best_fpath = best_model_dir + '/best_model.pt'
        shutil.copyfile(f_path, best_fpath)


def load_ckp(checkpoint_fpath, model, optimizer):
    checkpoint = torch.load(checkpoint_fpath)
    model.load_state_dict(checkpoint['state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer'])
    return model, optimizer, checkpoint['epoch'], checkpoint['best_acc']


def get_weighted_sampler(dataset):
    class_indices = {}
    for i in range(len(dataset)):
        _, label = dataset[i]
        if label not in class_indices:
            class_indices[label] = []
        class_indices[label].append(i)

    class_weights = {}
    for label in class_indices:
        class_weights[label] = 1 / len(class_indices[label])

    weights = [class_weights[label] for _, label in dataset]
    oversample_sampler = WeightedRandomSampler(weights, len(weights), replacement=True)

    return oversample_sampler


class ModelTrainer:

    def __init__(self, device, model, criterion, optimizer, dataset, validation_split=.2,
                 batch_size=32, shuffle_dataset=True,
                 random_seed=42, resume_from_checkpoint=False, project_name="Training", architecture="Unknown",
                 num_epochs=25, initial_lr=None, num_classes=None, weighted_sampler=False):

        os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
        os.environ['TORCH_USE_CUDA_DSA'] = '1'

        if resume_from_checkpoint:
            self.model, self.optimizer, self.start_epoch, self.best_acc = load_ckp(
                'checkpoint/checkpoint.pt',
                model, optimizer)
        else:
            self.model = model
            self.optimizer = optimizer
            self.start_epoch = 0
            self.best_acc = 0.0

        self.model.to(device)
        self.criterion = criterion

        self.dataset = dataset
        self.shuffle_dataset = shuffle_dataset
        self.random_seed = random_seed
        self.dataset_size = len(dataset)
        self.validation_split = validation_split
        self.device = device
        self.batch_size = batch_size
        self.lr_scheduler = None
        self.epochs = num_epochs
        self.nr_batch_report = 5
        self.num_classes = num_classes

        test_size = int(self.validation_split * len(dataset))
        train_size = len(dataset) - test_size
        train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

        if not weighted_sampler:
            train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size,
                                                       num_workers=0, drop_last=True)
        else:
            weighted_sampler = get_weighted_sampler(train_dataset)
            train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size,
                                                       sampler=weighted_sampler,
                                                       num_workers=0, drop_last=True)

        test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size,
                                                  num_workers=0, drop_last=True)

        self.dataloaders = {'train': train_loader,
                            'val': test_loader}

        self.dataset_sizes = {"train": train_size, "val": test_size}

        torch.backends.cudnn.benchmark = True

        torch.backends.cuda.matmul.allow_tf32 = True

        # The flag below controls whether to allow TF32 on cuDNN. This flag defaults to True.
        torch.backends.cudnn.allow_tf32 = True

        wandb.init(
            # set the wandb project where this run will be logged
            project=project_name,

            # track hyper parameters and run metadata
            config={
                "architecture": architecture,
                "epochs": num_epochs,
                "nr_batch_report": self.nr_batch_report,
                "initial_lr": initial_lr if initial_lr else "Unknown",
                "batch_size": self.batch_size
            }
        )

        wandb.log({"train_size": train_size, "test_size": test_size})

    def set_lr_scheduler(self, scheduler):
        self.lr_scheduler = scheduler

    def train_model(self):
        scaler = torch.cuda.amp.GradScaler()
        
        
        for epoch in range(self.start_epoch, self.epochs):
            # Each epoch has a training and validation phase
            wandb.log({
                "epoch": epoch
            })
            for phase in ['train', 'val']:
                torch.cuda.empty_cache()
                if phase == 'train':
                    self.model.train()  # Set model to training mode
                else:
                    self.model.eval()  # Set model to evaluate mode

                running_loss = 0.0
                running_corrects = 0

                # Iterate over data.
                y_true = []
                y_pred = []
                
                for batch_idx, (inputs, labels) in enumerate(self.dataloaders[phase]):
                    inputs = inputs.to(self.device)
                    labels = torch.as_tensor(labels).to(self.device)

                    # zero the parameter gradients
                    self.optimizer.zero_grad()

                    # forward
                    with torch.cuda.amp.autocast():
                        with torch.set_grad_enabled(phase == 'train'):
                            outputs = self.model(inputs)
                        if type(outputs) is tuple:
                            outputs = outputs[0]
                        loss = self.criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        scaler.scale(loss).backward()
                        scaler.step(self.optimizer)
                        scaler.update()

                        if batch_idx % self.nr_batch_report == 0:
                            wandb.log({"loss": loss.item()})
                            if self.lr_scheduler:
                                wandb.log({"lr": self.lr_scheduler.get_last_lr()[-1]})
                                self.lr_scheduler.step()

                    # statistics
                    running_loss += loss.item() * inputs.size(0)
                    _, preds = torch.max(outputs, 1)
                    y_pred.extend(preds.tolist())
                    y_true.extend(labels.tolist())

                epoch_acc = balanced_accuracy_score(y_true, y_pred)
                if phase == 'val':
                    wandb.log({
                        "epoch_acc_val": epoch_acc,
                    })
                else:
                    epoch_loss = running_loss / self.dataset_sizes[phase]
                    wandb.log({
                        "epoch_acc_train": epoch_acc,
                        "epoch_loss_train": epoch_loss,
                    })

                checkpoint = {
                    'epoch': epoch + 1,
                    'state_dict': self.model.state_dict(),
                    'optimizer': self.optimizer.state_dict(),
                    'best_acc': self.best_acc
                }

                # deep copy the model
                if phase == 'val' and epoch_acc > self.best_acc:
                    save_ckp(checkpoint, True, "checkpoint", 'best_model')
                    self.best_acc = epoch_acc
                    wandb.log({
                        "best_acc_val": self.best_acc
                    })
                save_ckp(checkpoint, False, "checkpoint", 'best_model')
        return self.model

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
criterion = nn.CrossEntropyLoss()
torch.manual_seed(123)
model_name = "dumitrescustefan/bert-base-romanian-uncased-v1"
dataset = SexismDataset(model_name, "dataset/train_data_final.csv")
model = SexismDetectionModel(model_name, num_classes=5)
optimizer = torch.optim.Adam(model.parameters(), lr=0.00001)

if not os.path.exists("best_model"):
    os.makedirs("best_model")
    
if not os.path.exists("checkpoint"):
    os.makedirs("checkpoint")

# if the program was interrupted we will resume from the last saved model
print('Type 0 to exit, 1 to start fresh, 2 from checkpoint')
inp = input()
if inp == '0':
    exit(0)

trainer = ModelTrainer(device, model, criterion, optimizer, dataset,
                        resume_from_checkpoint=(inp == '2'), project_name="Sexism NLP Hackathon",
                        architecture="Bert-Romanian SD", num_epochs=200, batch_size=256, initial_lr=1e-5,
                        validation_split=0.1,
                        num_classes=5, weighted_sampler=True)
trainer.train_model()

In [12]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model_name = "dumitrescustefan/bert-base-romanian-uncased-v1"
model = SexismDetectionModel(model_name, num_classes=5).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.00001)
model, optimizer, last_epoch, acc = load_ckp('best_model/best_model.pt', model, optimizer)
dataset = SexismDataset(model_name, 'dataset/test_data_final.csv', test=True)

dataloader = torch.utils.data.DataLoader(dataset, batch_size=16, shuffle=False, num_workers=0)

to_label = {
    0: 'direct',
    1: 'descriptive',
    2: 'reporting',
    3: 'offensive',
    4: 'non-offensive'
}

predictions = []
ids = [i for i in range(len(dataset))]
with torch.no_grad():
    for inp in dataloader:
        inp = inp.to(device)
        outputs = model(inp)
        if type(outputs) is tuple:
            outputs = outputs[0]
        _, preds = torch.max(outputs, 1)
        predictions.extend(preds.tolist())


pred = [to_label[pr] for pr in predictions]
output = pd.DataFrame({'Id': ids, 'Label': pred})
output.to_csv('submissionv15.csv', index=False)
print("Your submission was successfully saved!")

Some weights of the model checkpoint at racai/distilbert-base-romanian-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_projector.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at racai/distilbert-base-romanian-uncased and are newly initialized: ['classifier.bias', 'class

Your submission was successfully saved!
